In [80]:
# PreProcess Step
# TODO: While are NA values being represented when one hot encoding
# TODO: Deal with numerical values

import pandas as pd
from sklearn.linear_model import LinearRegression

file_path = 'train.csv'
data = pd.read_csv(file_path)

# Drop the 'Id' column in-place
data.drop('Id', axis=1, inplace=True)

# Numerical Preprocess
# Handling Numerical Data:
# Garage Year built has no data
# LotFrontage
# MasVnrArea
# GarageYrBlt

#################################################################################################
# Handles missing LotArea Data
# Prepare for 'LotFrontage' imputation
# Ensure no missing values in 'LotArea' if necessary
data['LotArea'].fillna(data['LotArea'].median(), inplace=True) # or any other appropriate method

# Fit model on non-missing 'LotFrontage'
non_na_data = data.dropna(subset=['LotFrontage'])
model = LinearRegression()
model.fit(non_na_data[['LotArea']], non_na_data['LotFrontage'])

# Predict and fill missing 'LotFrontage' values
missing_indices = data['LotFrontage'].isnull()
data.loc[missing_indices, 'LotFrontage'] = model.predict(data.loc[missing_indices, ['LotArea']])


# Remove entries with NA in MasVnrArea
# Unable to know what the difference between NA and 0 is in this column
# As only 8 entries have NA's for this one, decided to drop them
data = data.dropna(subset=['MasVnrArea'])

# HasGarage feature
data['HasGarage'] = data['GarageYrBlt'].apply(lambda x: 1 if x > 0 else 0)
# Replace NA values in 'GarageYrBlt' with zero, indicating no garage
data['GarageYrBlt'].fillna(0, inplace=True)
#################################################################################################

# Replacing missing values for numeric div columns with 0 (might consider using the median or mean)
numeric_cols = ['LotFrontage', 'LotArea']
data[numeric_cols] = data[numeric_cols].fillna(0)  # Adjust based on what's appropriate

# Replaces NA with nulls
data = data.fillna('')

# Feature Engineering
data['TotalSF'] = data['1stFlrSF'] + data['2ndFlrSF'] + data['TotalBsmtSF']
data['HouseAge'] = data['YrSold'] - data['YearBuilt']
data['RemodelAge'] = data['YrSold'] - data['YearRemodAdd']
data['HasBasement'] = data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
data['TotalBath'] = data['FullBath'] + (0.5 * data['HalfBath']) + data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath'])
data['OverallScore'] = data['OverallQual'] + data['OverallCond']
data['LotFrontageRatio'] = data['LotFrontage'] / data['LotArea']  # Ensure no division by zero


def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

data['SaleSeason'] = data['MoSold'].apply(get_season)

# Price per SF
# data['PricePerSF'] = data['SalePrice'] / data['TotalSF']


# Filling Empty Conditions per structure of data
# The columns where you want to fill missing values

#NOTE: MasVnrType should have no empty cells at this point as NA entries should have been dropped
columns_to_fill_na = [
    'Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
    'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 
    'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
    'MasVnrType', 'Electrical'
]

# Fill missing values with 'NA' only in the specified columns
data[columns_to_fill_na] = data[columns_to_fill_na].fillna('NA')

# Additonaly PreProcessing to get categorical data to binary
data = pd.get_dummies(data, columns=[
    'Alley', 'MSZoning', 'Street', 'LotShape',
    'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
    'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
    'Exterior2nd', 'Foundation', 'Heating', 'CentralAir',
    'Functional', 'GarageType', 'GarageFinish', 'PavedDrive',
    'MiscFeature', 'SaleType', 'SaleCondition', 'SaleSeason',
    'MasVnrType', 'Electrical'
])

# Label Encoding: Simplify Quality and Condition Ratings
# May not be suitable when dealing with neural networks?
# Blocking out and will test, will transform into one hot encoding
""""
quality_map = {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0}
data['ExterQual'] = data['ExterQual'].map(quality_map)
data['BsmtQual'] = data['BsmtQual'].map(quality_map)
data['ExterCond'] = data['ExterCond'].map(quality_map)
#'Heating', 'HeatingQC',

"""


data = pd.get_dummies(data, columns=[
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
    'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC',
    'Fence'
])


#missing NA features. i.e one hot encoded features that have no NA, are not here
missing_features = [
    'MSZoning_', 'KitchenQual_', 'Functional_', 
    'MasVnrType_', 'Exterior1st_', 'Utilities_', 
    'SaleType_', 'Exterior2nd_'
]

# Add each missing feature to the test set with all values set to 0
for feature in missing_features:
    data[feature] = 0



output_file_path_features = 'train_with_features_Gus.csv'
data.to_csv(output_file_path_features, index=False)

In [71]:
# PreProcess Step For TEST DATA
# We are dropping values, is there a way to do it without dropping values
# TODO: While are NA values being represented when one hot encoding
# TODO: Deal with numerical values

import pandas as pd
from sklearn.linear_model import LinearRegression

file_path = 'test.csv'
data = pd.read_csv(file_path)

# Drop the 'Id' column in-place
# data.drop('Id', axis=1, inplace=True)

# Numerical Preprocess
# Handling Numerical Data:
# Garage Year built has no data
# LotFrontage
# MasVnrArea
# GarageYrBlt

#################################################################################################
# Handles missing LotArea Data
# Prepare for 'LotFrontage' imputation
# Ensure no missing values in 'LotArea' if necessary
data['LotArea'].fillna(data['LotArea'].median(), inplace=True) # or any other appropriate method

# Fit model on non-missing 'LotFrontage'
non_na_data = data.dropna(subset=['LotFrontage'])
model = LinearRegression()
model.fit(non_na_data[['LotArea']], non_na_data['LotFrontage'])

# Predict and fill missing 'LotFrontage' values
missing_indices = data['LotFrontage'].isnull()
data.loc[missing_indices, 'LotFrontage'] = model.predict(data.loc[missing_indices, ['LotArea']])


# Remove entries with NA in MasVnrArea
# Unable to know what the difference between NA and 0 is in this column
# As only 8 entries have NA's for this one, decided to drop them
data = data.dropna(subset=['MasVnrArea'])

# HasGarage feature
data['HasGarage'] = data['GarageYrBlt'].apply(lambda x: 1 if x > 0 else 0)
# Replace NA values in 'GarageYrBlt' with zero, indicating no garage
data['GarageYrBlt'].fillna(0, inplace=True)
#################################################################################################

#Dropping TotalBsmtSF row NAs only 2 entries
data = data.dropna(subset=['TotalBsmtSF'])

#Dropping TotalBsmtSF row NAs only 2 entries
data = data.dropna(subset=['BsmtFullBath'])

#Dropping GarageCars row NAs only 1 entries
data = data.dropna(subset=['GarageCars'])        

#Dropping GarageArea row NAs only 1 entries
data = data.dropna(subset=['GarageArea'])  


# Replacing missing values for numeric div columns with 0 (might consider using the median or mean)
numeric_cols = ['LotFrontage', 'LotArea']
data[numeric_cols] = data[numeric_cols].fillna(0)  # Adjust based on what's appropriate

# Replaces NA with nulls
data = data.fillna('')

# Feature Engineering
data['TotalSF'] = data['1stFlrSF'] + data['2ndFlrSF'] + data['TotalBsmtSF']
data['HouseAge'] = data['YrSold'] - data['YearBuilt']
data['RemodelAge'] = data['YrSold'] - data['YearRemodAdd']
data['HasBasement'] = data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
data['TotalBath'] = data['FullBath'] + (0.5 * data['HalfBath']) + data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath'])
data['OverallScore'] = data['OverallQual'] + data['OverallCond']
data['LotFrontageRatio'] = data['LotFrontage'] / data['LotArea']  # Ensure no division by zero


def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

data['SaleSeason'] = data['MoSold'].apply(get_season)

# Filling Empty Conditions per structure of data
# The columns where you want to fill missing values

#NOTE: MasVnrType should have no empty cells at this point as NA entries should have been dropped
columns_to_fill_na = [
    'Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
    'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 
    'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
    'MasVnrType', 'Electrical'
]

# Fill missing values with 'NA' only in the specified columns
data[columns_to_fill_na] = data[columns_to_fill_na].fillna('NA')

# Additonaly PreProcessing to get categorical data to binary
data = pd.get_dummies(data, columns=[
    'Alley', 'MSZoning', 'Street', 'LotShape',
    'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
    'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
    'Exterior2nd', 'Foundation', 'Heating', 'CentralAir',
    'Functional', 'GarageType', 'GarageFinish', 'PavedDrive',
    'MiscFeature', 'SaleType', 'SaleCondition', 'SaleSeason',
    'MasVnrType', 'Electrical'
])

# Label Encoding: Simplify Quality and Condition Ratings
# May not be suitable when dealing with neural networks?
# Blocking out and will test, will transform into one hot encoding
""""
quality_map = {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0}
data['ExterQual'] = data['ExterQual'].map(quality_map)
data['BsmtQual'] = data['BsmtQual'].map(quality_map)
data['ExterCond'] = data['ExterCond'].map(quality_map)
#'Heating', 'HeatingQC',

"""


data = pd.get_dummies(data, columns=[
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
    'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC',
    'Fence'
])

missing_features = [
    'Condition2_RRAe', 'PoolQC_Fa', 'Heating_OthW', 'HouseStyle_2.5Fin', 
    'Condition2_RRAn', 'RoofMatl_Roll', 'Electrical_Mix', 'Heating_Floor', 
    'RoofMatl_Metal', 'RoofMatl_Membran', 'Condition2_RRNn', 
    'MiscFeature_TenC', 'RoofMatl_ClyTile', 'Exterior2nd_Other', 'Exterior1st_Stone', 
    'Utilities_NoSeWa', 'GarageQual_Ex', 'Exterior1st_ImStucc', 'Electrical_'
]

# Add each missing feature to the test set with all values set to 0
for feature in missing_features:
    data[feature] = 0



output_file_path_features = 'test_with_features_Gus.csv'
data.to_csv(output_file_path_features, index=False)


In [89]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import warnings
# Load the data
data = pd.read_csv('train_with_features_Gus.csv')
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

# Define a scaler and MLP regressor in a pipeline
# This ensures that scaling is correctly reapplied for each cross-validation fold
pipeline = make_pipeline(StandardScaler(), MLPRegressor(
    hidden_layer_sizes=(128, 64, 50),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    max_iter=1000,
    random_state=42
))

# Set up K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Calculate MSE across all folds
mse_scores = -cross_val_score(pipeline, X, y, cv=kfold, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(mse_scores)  # Convert MSE to RMSE

print("MSE scores for each fold:", mse_scores)
print("RMSE scores for each fold:", rmse_scores)
print("Average MSE:", np.mean(mse_scores))
print("Average RMSE:", np.mean(rmse_scores))


average_price = y.mean()
rmse_percentage = (rmse_scores.mean() / average_price) * 100
print(f"Average Sale Price: ${average_price:.2f}")
print(f"Average RMSE: ${rmse_scores.mean():.2f}")
print(f"RMSE as Percentage of Average Sale Price: {rmse_percentage:.2f}%")

# Compare with baseline RMSE
baseline_rmse = np.sqrt(mean_squared_error(y, [y.mean()] * len(y)))
print(f"Baseline RMSE: ${baseline_rmse:.2f}")
if rmse_scores.mean() < baseline_rmse:
    print("Model RMSE is better than baseline.")
else:
    print("Model RMSE is not better than baseline.")

#######################################################################################
# Testing: Load the preprocessed test data
test_data = pd.read_csv('test_with_features_Gus.csv')

# Assuming the first column is 'Id' and the rest are features used for prediction
X_test = test_data.drop('Id', axis=1)  # Drop 'Id' for prediction purposes

# Fit the pipeline to the full training data
pipeline.fit(X, y)

# Predict on the test data
y_test_pred = pipeline.predict(X_test)

# Create a DataFrame for submission that includes the Id and the predicted prices
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': y_test_pred
})

# Save the submission file
submission.to_csv('predictions.csv', index=False)

#######################################################################################
# Load the predictions and actual values
predictions_df = pd.read_csv('predictions.csv')
actual_df = pd.read_csv('sample_submission.csv')

# Sort both DataFrames by 'Id' to ensure alignment
predictions_df.sort_values('Id', inplace=True)
actual_df.sort_values('Id', inplace=True)

# Find mismatched IDs
predictions_ids = set(predictions_df['Id'])
actual_ids = set(actual_df['Id'])
mismatched_ids = actual_ids - predictions_ids

# Warn if there are mismatches and adjust the actual values DataFrame
if mismatched_ids:
    warnings.warn(f"Mismatched IDs found: {mismatched_ids}. These will be removed from the actuals.")
    actual_df = actual_df[actual_df['Id'].isin(predictions_ids)]

# Ensure the adjusted DataFrames are aligned correctly
actual_df.sort_values('Id', inplace=True)  # Re-sort to ensure order after adjustments


# Calculate the logarithm of predictions and actual values to prevent scale bias
log_predictions = np.log(predictions_df['SalePrice'] + 1)
log_actual = np.log(actual_df['SalePrice'] + 1)

# Calculate RMSE using the log-transformed values
mse = mean_squared_error(log_actual, log_predictions)
rmse = np.sqrt(mse)

print(f'Logarithmic RMSE: {rmse}')


MSE scores for each fold: [  1.14383829e+09   1.41103947e+09   1.62383361e+09   3.51772686e+09
   1.11542797e+09]
RMSE scores for each fold: [ 33820.6784853   37563.80531182  40296.81883646  59310.42793354
  33398.02350739]
Average MSE: 1762373241.38
Average RMSE: 40877.9508149
Average Sale Price: $180615.06
Average RMSE: $40877.95
RMSE as Percentage of Average Sale Price: 22.63%
Baseline RMSE: $79258.23
Model RMSE is better than baseline.
Logarithmic RMSE: 3.0168102385336213


/usr/lib/python3/dist-packages/ipykernel_launcher.py:94: UserWarning: Mismatched IDs found: {2369, 2593, 2658, 2341, 2312, 1993, 2121, 1707, 2189, 2350, 2863, 2577, 2005, 2326, 2042, 1883, 1692, 2687}. These will be removed from the actuals.
